In [2]:
cd ..

d:\project\cls_product


In [ ]:
from datasets import load_dataset

food = load_dataset("food101",split="train[:5000]")

c:\Users\Moaaz Reda\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Moaaz Reda\.cache\huggingface\hub\datasets--food101. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downloading from https://cdn-lfs.hf.co/datasets/food101/e45129fa1cb069859df21467b68

HfHubHTTPError: 403 Forbidden: None.
Cannot access content at: https://cdn-lfs.hf.co/datasets/food101/e45129fa1cb069859df21467b68bab7e57fcb0d93c9cf36c7e4bc56d3cdddc0b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train-00006-of-00008.parquet%3B+filename%3D%22train-00006-of-00008.parquet%22%3B&Expires=1754649553&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NDY0OTU1M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9kYXRhc2V0cy9mb29kMTAxL2U0NTEyOWZhMWNiMDY5ODU5ZGYyMTQ2N2I2OGJhYjdlNTdmY2IwZDkzYzljZjM2YzdlNGJjNTZkM2NkZGRjMGI~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=FvXcFWslJ9rZnG~uV-vxjp2md8q3wRNGyx52r9jHGpUJJKP6TWLvSXVZb8G--TC~pQKcQenz15WlLab2wPz1jiNpriQWO826qG6Erl33T3KacD9~c6S8xbdWk4DUjfup9yGUr~m5~DK8c2m5KN7lmOBrywF2tPsm4VgL-MlT4tGgyfgQuI25vC7v-LJhqQ-Z91K7ScZJMAho1FqSabaa5cNHqHsORYPW6CFkOKQoSRIb72KLO9fJTQvHLKL5Mr4EiCddWP2Fw4ij2EylcSS3DswFJZMoZ0pbB-7pgmhag~gdZPGI454-bCvOxEBnl1uSGottdhpJ27SQiWHy9-2Twg__&Key-Pair

In [ ]:
food_split = food.train_test_split(0.2)

In [ ]:
labels = food_split['train'].features['label'].names
label2id, id2label = {}, {}
for i , label in enumerate(labels):
    label2id[label] = i,
    id2label[id] = label

In [ ]:
from transformers import AutoImageProcessor 

processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
size = (processor.size['height'],processor.size['width'])

In [ ]:
from torchvision.transform import RandomResizedCrop ,Compos ,Resize ,Normalize ,ToTensor

In [ ]:
transform = Compos([
    RandomResizedCrop(size),
    ToTensor(),
    Normalize(mean=processor.image_mean,std=processor.image_std)
])

In [ ]:
def process_image(samples):
    samples['pexil_value'] = [transform(img.convert(RGB)) for img in samples['image']]
    del samples['image']
    return samples

In [ ]:
food_split_proc = food_split.with_transform(process_image)

In [ ]:
from transformers import AutoModelForImageClassification ,TrainingArguments ,Trainer,default_data_collator
import evaluate
import numpy as np
data_collator = default_data_collator
metric = evaluate.load('accuracy')

In [ ]:
def compute_metrics(predictions_and_labels):
    predictions,labels = predictions_and_labels
    predictions = np.argmax(predictions,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [ ]:
model = AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',num_labels=len(labels),id2label=id2label,label2id=label2id)

In [ ]:
training_args = TrainingArguments(
    # output_dir="my_model",
    output_dir="./results",
    report_to=[],
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    weight_decay=0.01,
    remove_unused_columns=False,
    save_strategy="epoch",
    learning_rate=0.001,
    num_train_epochs=7,
    logging_steps=10,
    push_to_hub=False,
    metric_for_best_model="accuracy"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=food_split_proc['train'],
    eval_dataset=food_split_proc['test'],
    processing_class=processor
)

In [ ]:
trainer.train()